## EXPLAINATION

- converting to the format usable with CAUSE paper [CAUSE: Learning Granger Causality from Event Sequences using Attribution Methods]
    - input:
        - a dataframe from [C:\DATA\data\raw\physionet_challenge_2019], cols: id - {vitals} - {labs} - {demo} - {sepsis shcok}
    - descritize to range [L,N,H]|[-1,0,1]
    - take difference
    - extract these features for {vitals} - {labs}
        - N2H, N2L, sample time (for some of labs only)
    - output:
        - a dataframe writtern to [C:\DATA\data\raw\physionet_challenge_2019\physio_CAUSE.csv], cols: id-{N2H}-{N2L}-{sample times}
        

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots



import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning import Trainer

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from pytorch_lightning.loggers import TensorBoardLogger

import optuna
from optuna.integration import PyTorchLightningPruningCallback

ADD_DATA = "C:\\DATA\\data\\raw\\physionet_challenge_2019\\"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


c:\Users\hokarami\Anaconda3\envs\dashVis\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\hokarami\Anaconda3\envs\dashVis\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
df_train = pd.read_csv(ADD_DATA+'df_AA.csv')
df_valid = pd.read_csv(ADD_DATA+'df_BB.csv')
df = df_train

In [ ]:
df.columns
df.dtypes.values
df.describe()


## settings

In [2]:
cols_vs = ['HR', 'O2Sat', 'Temp', 'MAP', 'Resp']
cols_lab = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets',]

df_temp = pd.read_csv(ADD_DATA+'ranges.csv')
#df_temp

range_vs = {}
for col in cols_vs:
       q=df_temp[df_temp['label']==col]
       range_vs[col]=[-1e20, q.iloc[0]['ref_range_lower'] , q.iloc[0]['ref_range_upper'], 1e20 ]

range_lab = {}
for col in cols_lab:
       q=df_temp[df_temp['label']==col]
       range_lab[col]=[-1e20, q.iloc[0]['ref_range_lower'] , q.iloc[0]['ref_range_upper'], 1e20 ]


# pd.DataFrame(cols_lab).to_csv(ADD_DATA+'ranges.csv')


## Functions

In [ ]:
def fun_desc_diff (df_sub):
    # STEP1: discretizing each variable into 3 ranges [L,N,H] == [-1,0,1]
    for col in cols_vs:
        df_sub[col+'_desc'] = pd.cut(df_sub[col], bins=range_vs[col], labels=['L','N','H'])
        df_sub[col+'_desc'] = df_sub[col+'_desc'].cat.codes-1
        df_sub.loc[ df_sub[col].isnull(),  col+'_desc'  ] = np.nan
    for col in cols_lab:
        df_sub[col+'_desc'] = pd.cut(df_sub[col], bins=range_lab[col], labels=['L','N','H'])
        df_sub[col+'_desc'] = df_sub[col+'_desc'].cat.codes-1
        df_sub.loc[ df_sub[col].isnull(),  col+'_desc'  ] = np.nan


    # STEP2: define new features based on descretized values
    for col in [*cols_vs, *cols_lab]:
        temp = df_sub[col+'_desc'].ffill().diff().replace(0,np.nan)
        df_sub.loc[ (temp==1) & (df_sub[col+'_desc']==1 ),col+'_N2H'] = 0
        df_sub.loc[ (temp==-1) & (df_sub[col+'_desc']==-1),col+'_N2L'] = 0


    return df_sub
    



# fig = px.line(df_temp, x='times', y=cols_vs)
# fig.show()
#df_temp~~



## MAIN PROCESS

In [59]:
df = pd.read_csv(ADD_DATA+'df_AA.csv')
df = df[['id','SepsisLabel', *cols_vs,*cols_lab]]
ids = df['id'].unique().tolist()

df.columns
df.describe()

Index(['id', 'SepsisLabel', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BaseExcess',
       'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos',
       'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose',
       'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total',
       'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets'],
      dtype='object')

,id,SepsisLabel,HR,O2Sat,Temp,MAP,Resp,BaseExcess,HCO3,FiO2,...,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets
count,388467.000000,388467.000000,358125.000000,341463.000000,132301.000000,348451.000000,350108.000000,41255.000000,31141.000000,55306.000000,...,19376.000000,42489.000000,4602.000000,476.000000,46107.000000,34466.000000,18577.000000,29108.000000,2778.000000,25198.000000
mean,5001.655477,0.021317,85.118149,97.271563,37.031737,78.635414,18.803352,-0.704236,24.106756,0.527347,...,3.589175,4.164884,2.438451,9.726681,30.715460,10.594408,40.816265,11.949978,289.309647,200.401826
std,2891.352092,0.144439,16.915598,2.899507,0.789902,14.972182,5.396067,4.297338,4.399595,0.186646,...,1.450509,0.644397,4.550178,11.780629,4.890605,1.747932,24.136283,6.965158,156.551527,109.786879
min,0.000000,0.000000,20.000000,22.000000,20.900000,20.000000,1.000000,-32.000000,0.000000,0.000000,...,0.300000,1.800000,0.100000,0.300000,8.800000,3.100000,17.100000,0.100000,56.000000,5.000000
25%,2482.000000,0.000000,73.000000,96.000000,36.560000,68.000000,15.000000,-3.000000,22.000000,0.400000,...,2.700000,3.800000,0.500000,0.900000,27.400000,9.400000,27.700000,8.100000,181.000000,128.000000
50%,4984.000000,0.000000,84.000000,98.000000,37.060000,77.000000,18.000000,0.000000,24.000000,0.500000,...,3.400000,4.100000,0.900000,4.450000,30.200000,10.400000,32.400000,10.900000,246.000000,182.000000
75%,7510.000000,0.000000,96.000000,99.500000,37.560000,87.000000,22.000000,1.000000,27.000000,0.600000,...,4.200000,4.500000,2.200000,13.725000,33.600000,11.600000,42.800000,14.300000,355.000000,246.000000
max,9999.000000,1.000000,280.000000,100.000000,41.600000,300.000000,69.000000,49.500000,55.000000,7.000000,...,17.600000,27.500000,45.900000,49.000000,71.700000,22.100000,150.000000,224.900000,1760.000000,1783.000000


In [60]:
# adding time column for each subject
df['dummy']=1
df['t'] = df.groupby('id')['dummy'].cumsum()




for col in cols_vs:
        df[col+'_desc'] = pd.cut(df[col], bins=range_vs[col], labels=['L','N','H'])
        df[col+'_desc'] = df[col+'_desc'].cat.codes-1
        df.loc[ df[col].isnull(),  col+'_desc'  ] = np.nan
for col in cols_lab:
    df[col+'_desc'] = pd.cut(df[col], bins=range_lab[col], labels=['L','N','H'])
    df[col+'_desc'] = df[col+'_desc'].cat.codes-1
    df.loc[ df[col].isnull(),  col+'_desc'  ] = np.nan

# cols_desc = [col for col in df.columns if '_desc' in col ]
# cols_N2H = [col+'_N2H' for col in cols_desc]
# cols_N2L = [col+'_N2L' for col in cols_desc]


# df[cols_N2H] = df.groupby('id')[cols_desc].ffill().diff().replace(0,np.nan)

for col in [*cols_vs, *cols_lab]:
    temp = df.groupby('id')[col+'_desc'].ffill().diff().replace(0,np.nan)

    df[col+'_N2H'] = np.nan
    df.loc[ (temp==1) & (df[col+'_desc']==1 ),col+'_N2H'] = 0   # if current value is high and there is increase

    df[col+'_N2L'] = np.nan
    df.loc[ (temp==-1) & (df[col+'_desc']==-1 ),col+'_N2L'] = 0   # if current value is low and there is decrease

    df[col+'_sam'] = np.nan
    df.loc[ (df[col].notnull() ),col+'_sam'] = 0   # if value has measured




#df.loc[df['SepsisLabel']==0,'SepsisLabel']=np.nan



temp = df.groupby('id')['SepsisLabel'].ffill().diff()


df['SepsisLabel']=np.nan
df.loc[ (temp==1) ,'SepsisLabel'] = 1   # if current value is high and there is increase
df['SepsisLabel'] = df['SepsisLabel'].replace(0,np.nan)



df['id_diff'] = df['id'].diff()
df.loc[df['id_diff']==0, 'id_diff']=np.nan

# # STEP2: define new features based on descretized values
# for col in [*cols_vs, *cols_lab]:
#     temp = df[col+'_desc'].ffill().diff().replace(0,np.nan)
#     df.loc[ (temp==1) & (df[col+'_desc']==1 ),col+'_N2H'] = 0
#     df.loc[ (temp==-1) & (df[col+'_desc']==-1),col+'_N2L'] = 0

# 



C:\Users\hokarami\AppData\Local\Temp\ipykernel_23056\37798464.py:33: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\hokarami\AppData\Local\Temp\ipykernel_23056\37798464.py:27: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\hokarami\AppData\Local\Temp\ipykernel_23056\37798464.py:30: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragm

In [61]:
df.describe()
df.columns


,id,SepsisLabel,HR,O2Sat,Temp,MAP,Resp,BaseExcess,HCO3,FiO2,...,WBC_N2H,WBC_N2L,WBC_sam,Fibrinogen_N2H,Fibrinogen_N2L,Fibrinogen_sam,Platelets_N2H,Platelets_N2L,Platelets_sam,id_diff
count,388467.000000,857.0,358125.000000,341463.000000,132301.000000,348451.000000,350108.000000,41255.000000,31141.000000,55306.000000,...,0.0,0.0,29108.0,0.0,61.0,2778.0,0.0,0.0,25198.0,9999.0
mean,5001.655477,1.0,85.118149,97.271563,37.031737,78.635414,18.803352,-0.704236,24.106756,0.527347,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0
std,2891.352092,0.0,16.915598,2.899507,0.789902,14.972182,5.396067,4.297338,4.399595,0.186646,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0
min,0.000000,1.0,20.000000,22.000000,20.900000,20.000000,1.000000,-32.000000,0.000000,0.000000,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0
25%,2482.000000,1.0,73.000000,96.000000,36.560000,68.000000,15.000000,-3.000000,22.000000,0.400000,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0
50%,4984.000000,1.0,84.000000,98.000000,37.060000,77.000000,18.000000,0.000000,24.000000,0.500000,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0
75%,7510.000000,1.0,96.000000,99.500000,37.560000,87.000000,22.000000,1.000000,27.000000,0.600000,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0
max,9999.000000,1.0,280.000000,100.000000,41.600000,300.000000,69.000000,49.500000,55.000000,7.000000,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0


Index(['id', 'SepsisLabel', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BaseExcess',
       'HCO3', 'FiO2',
       ...
       'WBC_N2H', 'WBC_N2L', 'WBC_sam', 'Fibrinogen_N2H', 'Fibrinogen_N2L',
       'Fibrinogen_sam', 'Platelets_N2H', 'Platelets_N2L', 'Platelets_sam',
       'id_diff'],
      dtype='object', length=160)

In [58]:
# saving

df.to_csv(ADD_DATA+'physio2019_CAUSE.csv')

## vis for one subject

In [61]:
# select a random or specified subject
import random
id = random.choice(df['id'])
id
id=896
df[df['id']==id].shape

los = df.groupby('id').size().tolist()

#px.histogram(los)

df_sub = df[df['id']==id]

df_sub.columns


346

(221, 156)

Index(['id', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BaseExcess', 'HCO3',
       'FiO2', 'pH',
       ...
       'Hgb_N2H', 'Hgb_N2L', 'PTT_N2H', 'PTT_N2L', 'WBC_N2H', 'WBC_N2L',
       'Fibrinogen_N2H', 'Fibrinogen_N2L', 'Platelets_N2H', 'Platelets_N2L'],
      dtype='object', length=156)

In [62]:
my_list = [*cols_vs, *cols_lab]
#my_list = ['Glucose', 'HR', 'Temp']
df_temp = df_sub
df_temp.times = np.arange(len(df_sub))


fig = make_subplots(rows=len(my_list), cols=4, shared_xaxes=True)
j = 1
for i,col in enumerate(my_list):
    fig.append_trace(
        go.Scatter(
            {'x': df_temp.times, 
             'y': df_temp[col]},
             name = col,
              mode = 'markers'
             ),
              
             row=i+1, col=1)
    #_ = fig.add_hrect(y0=range_vs[col][0][1], y1=range_vs[col][0][-2], line_width=0, fillcolor="green", opacity=0.2,row=i+1, col=1)
    
    fig.append_trace(
        go.Scatter(
            {'x': df_temp.times, 
             'y': df_temp[col+'_desc'] },
             name = col+'_desc',
             mode = 'markers'
             ),
             
              
             row=i+1, col=2)
    _ = fig.update_yaxes(range=[-1.1, 1.1], row=i+1, col=2)



    # fig.append_trace(
    #     go.Scatter(
    #         {'x': df_temp.times, 
    #          'y': df_temp[col+'_desc'].ffill().diff().replace(0,np.nan) },
    #          name = col+'_desc',
    #          mode = 'markers'
    #          ),
              
    #          row=i+1, col=3)
    # _ = fig.update_yaxes(range=[-1.1, 1.1], row=i+1, col=3)

    fig.append_trace(
        go.Scatter(
            {'x': df_temp.times, 
             'y': df_temp[col+'_N2H'] },
             name = col+'_N2H',
             mode = 'markers'
             ),
              
             row=i+1, col=3)

    fig.append_trace(
        go.Scatter(
            {'x': df_temp.times, 
             'y': df_temp[col+'_N2L'] },
             name = col+'_N2L',
             mode = 'markers'
             ),
              
             row=i+1, col=4)

_ = fig.update_layout(height=3000, width=1000, title_text="Stacked Subplots")

fig.show()

C:\Users\hokarami\AppData\Local\Temp\ipykernel_1548\3376927795.py:4: UserWarning:

Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access

